In [2]:
import pulp
from random import choice

# Data

In [34]:
# users
users = ["u_%s" % k for k in range(50)]
positions = [(choice(range(10)), choice(range(10))) for k in range(len(users))]
position_user=dict(zip(users, positions))
speed=[choice(range(5,10)) for k in range(len(users))]
speed_user=dict(zip(users, speed))
latency=[choice(range(3)) for k in range(len(users))]
latency_user=dict(zip(users, latency))

# access points
access_points=["a_%s" % k for k in range(8)]
_range=[choice(range(5)) for k in range(len(access_points))]
range_access_point=dict(zip(access_points, _range))
speed=[choice(range(10)) for k in range(len(access_points))]
speed_access_point=dict(zip(access_points, speed))

# positions
positions=[(i, j) for i in range(10) for j in range(10)]

# distances
dist={}
for u in users:
    for p in positions:
        dist[(u, p)]=abs(position_user[u][0] - p[0]) + \
              abs(position_user[u][1] - p[1])


# costs
# liaisons = [(c, e) for c in clients for e in entrepots_candidats]
c = {}
for u in users:
    for a in access_points:
        c[(u,a)] = {}
        for p in positions:
            if dist[(u,p)] > range_access_point[a]:
                c[(u,a)][p] = 0
            else:
                c[(u,a)][p] = speed_access_point[a]*speed_user[u]-latency_user[u]          

# MIP

In [35]:
# create problem
prob = pulp.LpProblem("access_points", pulp.LpMaximize)

# define variables
x = pulp.LpVariable.dicts("x",
                          (users, access_points, positions),
                          cat=pulp.LpBinary)
z = pulp.LpVariable.dicts("x", (access_points, positions),
                          cat=pulp.LpBinary)

# objective functions
prob += pulp.lpSum([
    c[(u, a)][p] * x[u][a][p] for u in users for a in access_points for p in positions
])

# one access point per user
for u in users:
    prob += pulp.lpSum([x[u][a][p]
                       for a in access_points for p in positions]) == 1

# one position per access point
for a in access_points:
    prob += pulp.lpSum([z[a][p] for p in positions]) == 1
    
# at most one access point per position
for p in positions:
    prob += pulp.lpSum([z[a][p] for a in access_points]) <= 1
    
# variable consistency
for u in users:
    for a in access_points:
        for p in positions:
            prob += x[u][a][p] <= z[a][p]

In [36]:
prob.solve()

1

In [42]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=[position_user[k][0] for k in position_user],
        y=[position_user[k][1] for k in position_user],
        mode="markers",
        name="users",
    ))

for u in users:
    for a in access_points:
        for p in positions:
            if pulp.value(x[u][a][p]) > 0.9:
                fig.add_trace(
                    go.Scatter(x=[position_user[u][0], p[0]],
                               y=[position_user[u][1], p[1]],
                               mode='lines',
                               name='lines',
                               line=dict(color="green"),
                               showlegend=False))
fig.add_trace(go.Scatter(x=[p[0] for p in z[a] for a in z if pulp.value(z[a][p]) > 0.9],
                         y=[p[1] for p in z[a]
                             for a in z if pulp.value(z[a][p]) > 0.9],
                         mode="markers",
                         name="access_points",
                         marker=dict(color="red"),
                         marker_size=20))

fig.update_layout(
    yaxis=dict(
        tickmode='linear',
        tick0=0,
        dtick=1
    ),
    xaxis=dict(
        tickmode='linear',
        tick0=0,
        dtick=1
    )
)
fig.show()